In [63]:
import pandas as pd
import psycopg2
%matplotlib inline

In [51]:
conn = psycopg2.connect('host=localhost dbname=airbnb-analysis')

In [69]:
query = """
SELECT
listings.id as airbnb_id,
listings.listing_url as airbnb_url,
listings.host_total_listings_count as airbnb_listings_by_host,
listings.is_location_exact as airbnb_location_exact,
listings.room_type as airbnb_room_type,
rso."APN" as rso_apn,
rso."Secondary_Address" as rso_address,
rso."Category" as rso_type,
ST_Distance(geography(rso.geom), geography(listings.geom)) AS distance
FROM
listings 
CROSS JOIN LATERAL
(SELECT merged."APN", 
        merged.geom, 
        merged."Secondary_Address", 
        merged."Category"
FROM merged
ORDER BY
listings.geom <-> geom
LIMIT 1) AS rso
ORDER BY distance;
"""

In [70]:
df = pd.read_sql(query, conn)
total_scraped = len(df)
df.head(5)

,airbnb_id,airbnb_url,airbnb_listings_by_host,airbnb_location_exact,airbnb_room_type,rso_apn,rso_address,rso_type,distance
0,12053638,https://www.airbnb.com/rooms/12053638,2.0,True,Private room,4302018031,8933 W CADILLAC AVE LOS ANGELES 90034,MULTIFAMILY,0.423321
1,12592466,https://www.airbnb.com/rooms/12592466,1.0,True,Entire home/apt,2353012036,5045 N FAIR AVE LOS ANGELES 91601,CONDOMINIUM,0.475009
2,6800231,https://www.airbnb.com/rooms/6800231,9.0,True,Shared room,4265008063,11759 W MAYFIELD AVE LOS ANGELES 90049,CONDOMINIUM,0.490954
3,3731479,https://www.airbnb.com/rooms/3731479,1.0,False,Entire home/apt,4238011022,1307 S INNES PL LOS ANGELES 90291,MULTIFAMILY,0.545337
4,1056743,https://www.airbnb.com/rooms/1056743,1.0,True,Entire home/apt,5419017002,1537 N LAKE SHORE AVE LOS ANGELES 90026,MULTIFAMILY,0.585486


## Filtering
* Only whole rooms 
* Only exact locations 

In [73]:
df.airbnb_room_type.value_counts()

Entire home/apt    15184
Private room        9597
Shared room         1299
Name: airbnb_room_type, dtype: int64

In [74]:
df.airbnb_location_exact.value_counts()

True     21370
False     4710
Name: airbnb_location_exact, dtype: int64

In [75]:
df = df[df['airbnb_room_type'] == 'Entire home/apt']

In [76]:
df = df[df['airbnb_location_exact'] == True]

In [77]:
print("Total number of qualified listings: %s" % str(len(df)))
print("Total number of scraped listings: %s" % str(total_scraped))

Total number of qualified listings: 12859
Total number of scraped listings: 26080


^ is the number of homes that have an exact location on AirBNB and are an entire home/apt. Distance is in Meters. 

_Here are the most likely candidates. _ 

In [79]:
df.head(5)

,airbnb_id,airbnb_url,airbnb_listings_by_host,airbnb_location_exact,airbnb_room_type,rso_apn,rso_address,rso_type,distance
1,12592466,https://www.airbnb.com/rooms/12592466,1.0,True,Entire home/apt,2353012036,5045 N FAIR AVE LOS ANGELES 91601,CONDOMINIUM,0.475009
4,1056743,https://www.airbnb.com/rooms/1056743,1.0,True,Entire home/apt,5419017002,1537 N LAKE SHORE AVE LOS ANGELES 90026,MULTIFAMILY,0.585486
5,2739290,https://www.airbnb.com/rooms/2739290,3.0,True,Entire home/apt,5401005020,627 N TULAROSA DR LOS ANGELES 90026,MULTIFAMILY,0.586285
6,9347680,https://www.airbnb.com/rooms/9347680,1.0,True,Entire home/apt,5427005014,922 N MANZANITA ST LOS ANGELES 90029,MULTIFAMILY,0.673489
9,14088884,https://www.airbnb.com/rooms/14088884,2.0,True,Entire home/apt,4303002047,1503 S SHENANDOAH ST LOS ANGELES 90035,MULTIFAMILY,0.808627


In [ ]:
df.to_csv('data/likelyh')